In [ ]:
##NOTES##
#Extract the data in raw format first. Don't replace unwanted characters during scraping. Only replace after scraping as there can be new permutations of unwanted characters that will impede the scraping process.


#Relationships to check
#1 premium ad = faster sale?
#2 direct owner = lower depre?
#3 depre lower than usual = faster sale? - have to calculate for those missing depre
#4 more exclamation marks in Features, Accessories and Description fields = faster sale?
#5 lower no. of owners = higher depre?
#6 lower mileage = higher depre?
#7 direct owner = slower sale? - by virtue of schedule limitations vs a car sitting in a dealership, available for viewing for much longer hours

#KEY FEATURES OF CODE
#Mechanism to update availability of car from 'Available' to 'Sold'
#List of valid vehicle ids such that non-valid ones do not get scraped

# print(page_html)


In [ ]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from time import time
import random
import sqlite3
import re
from datetime import datetime
import datetime as dt

# #page in mutliples of 100 start from 0 i.e. 0, 100, 200, 300... cumulative count of vehicles
# id_first = 757059
# id_last = 778322

# car_id = 779360
# url = 'https://www.sgcarmart.com/used_cars/info.php?ID=' + str(car_id)

# response = get(url)
# page_html = BeautifulSoup(response.text,'html.parser')


In [ ]:
# Next we define a function process_new() which extracts the data for a single vehicle based on its ID (car_id).
# Later on, we will run this function over a list of multiple vehicles.

def process_new(car_id):
    # Fortunately the URL of a listing contain the ID of the car. What we need to do here is to simply alter the unique ID at the end of the URL.
    # We then use bs4's BeautifulSoup() function to extract the raw html of the webpage. We also extract the title of the website as this will be useful in filtering out non-valid IDs.

    url = 'https://www.sgcarmart.com/used_cars/info.php?ID=' + car_id
    response = get(url)
    page_html = BeautifulSoup(response.text,'html.parser')
    page_title = page_html.find('title').text
    
    # Next we grab from the full html a section (div class) called main_left. This section corresponds to the area of the webpage that contains all of the key data that we will be scraping later on. We need main_left for the next step.
    
    try:
        main_left = page_html.find('div', attrs = {'id':'main_left'})
     ###   tr_list = main_left.find_all('tr')
    except:
        None
    
    # Now an if statement is used to include only vehicles that are available for sale. 
    # The first page_title condition filters out non-valid IDs which when accessed, returns a webpage with the title (page_title) 'Buy Used Car & Used Vehicle & Used Cars Singapore - sgCarMart'.
    # The second condition searches for listings for which the vehicles are available. Sold vehicles and expired listings are hence excluded.

    if(page_title != 'Buy Used Car & Used Vehicle & Used Cars Singapore - sgCarMart' and
    str(main_left.find('strong', text = 'Availability').find_parent('tr').find_all('td')[1].text.replace('\n','')) == 'Available'):
        
        # We then scrape the various data of the vehicles that are available for sale. To clean up the formatting of numerical data, we either used a series of quick and dirty .replace() functions, or used Regex for items that had more variations in formatting. 
        
        # 1. Availability - now set as Available and may be updated to 'Sold' in the future by the subsequent update() function.
        avail = str(main_left.find('strong', text = 'Availability').find_parent('tr').find_all('td')[1].text.replace('\n',''))

        # 2. Make and Model - e.g. Toyota Vios
        ### mm_repl_list = ['New','10-yr','COE',' (  )', '5-yr']
        make_model = page_html.find('a', class_ = 'link_redbanner').text
        ### for item in mm_repl_list:
        ###     make_model = make_model.replace(item, '')

        # 3. Price
        try:
            price = int(main_left.find('strong', text = 'Price').find_parent('tr').find_all('strong')[1].text.replace('$','').replace(',',''))
        except:
            price = 0
            
        # 4. Annual Depreciation
        ### depre_repl_list = ['$', ',', ' /yr']
        try:
            depre = int(main_left.find('strong', text = 'Depreciation').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',','').replace(' /yrView models with similar depreciation',''))
        except:
            depre = 0
        ### for item in depre_repl_list:
        ###     depre = depre.replace(item,'')
        ### depre = int(depre)

        # 5. Registration Date
        ### regdate_repl_list = 
        try:
            reg_date = main_left.find('strong', text = 'Reg Date').find_parent('tr').find_all('td')[1].text
            reg_date = re.findall(r"\d+.+\(", reg_date)[0].replace('(','')
            reg_date = datetime.strptime(reg_date,'%d-%b-%Y')
        except:
            reg_date = 0

        # 6. Year of Manufacture 
        try:
            manu_yr = int(main_left.find('strong', text = 'Manufactured').find_parent('tr').find_all('td')[1].text)
        except:
            manu_yr = 0
            
        # 7. Mileage (in kilometres)
        try:
            mileage = main_left.find('strong', text = 'Mileage').find_parent('tr').find_all('td')[1].text
            mileage = int(re.findall(r"\d+.\d+ k", mileage)[0].replace(' k','').replace(',',''))
        except:
            mileage = 0
            
        # 8. Transmission - Auto or Manual
        try:
            tranny = main_left.find('strong', text = 'Transmission').find_parent('tr').find_all('td')[1].text
        except:
            tranny = ''
            
        # 9. Engine Capacity (in cc)
        try:
            eng_cap = int(main_left.find('strong', text = 'Engine Cap').find_parent('tr').find_all('td')[1].text.replace(',','').replace(' cc',''))
        except:
            eng_cap = 0
            
        # 10. Road Tax / Year
        try:
            road_tax = int(main_left.find('strong', text = 'Road Tax').find_parent('tr').find_all('td')[1].text.replace(',','').replace('$','').replace(' /yr',''))
        except:
            road_tax = ''

        # 11. Power (in bhp)
        try:
            power = main_left.find('strong', text = 'Power').find_parent('tr').find_all('td')[1].text
            power = int(re.findall(r"\d+ bhp", power)[0].replace(' k','').replace(' bhp',''))
        except:
            power = ''

        # 12. Curb Weight (in kg)
        try:
            weight = int(main_left.find('strong', text = 'Curb Weight').find_parent('tr').find_all('td')[1].text.replace(',','').replace(' kg',''))
        except:
            weight = 0
            
        # 13. Features - descriptive field, e.g. 'reverse camera and dual zone climate control'; chunk of text and not tags
        try:
            features = main_left.find('strong', text = 'Features').find_parent('tr').find_all('td')[1].text
        except:
            features = ''
            
        # 14. Accessories - descriptive field, e.g. 'carbon fibre spoiler and custom bodykit'; chunk of text and not tags
        try:
            acc = main_left.find('a', text = 'Accessories').find_parent('tr').find_all('td')[1].text
        except:
            acc = ''
            
        # 15. Description - descriptive field, e.g. 'accident free and parked under shelter'; chunk of text and not tags
        try:
            desc = main_left.find('strong', text = 'Description').find_parent('tr').find_all('td')[1].text
        except:
            desc = ''
            
        # 16. COE Period Left (in days) - number of days in a month taken to be 30
        try:
            coe_left = main_left.find('strong', text = 'Reg Date').find_parent('tr').find_all('td')[1].text
            try:
                yrs = int(re.findall(r"\d+(?=yr)",coe_left)[0])
            except:
                yrs = 0
            try:
                mths = int(re.findall(r"\d+(?=mth)",coe_left)[0])
            except:
                mths = 0
            try:
                days = int(re.findall(r"\d+(?=day)",coe_left)[0])
            except:
                days = 0
            coe_left = yrs * 365 + mths * 30 + days
        except:
            coe_left= 0
            
        # 17. COE Price Paid
        try:
            coe_price = int(main_left.find('a', text = 'COE').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',',''))
        except:
            coe_price = 0
            
        # 18. Open Market Value (OMV)
        try:
            omv = int(main_left.find('strong', text = 'OMV').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',',''))
        except:
            omv = 0
            
        # 19. Additional Registration Fee (ARF)
        try:
            arf = int(main_left.find('strong', text = 'ARF').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',',''))
        except:
            arf = 0
            
        # 20. Number of Past Owners
        try:
            owners = int(main_left.find('strong', text = 'No. of Owners').find_parent('tr').find_all('td')[1].text)
        except:
            owners = 0
            
        # 21. Vehicle Type - e.g. 'Sports Car', 'Luxury Sedan'. Each car can only have one vehicle type.
        try:
            veh_type = main_left.find('strong', text = 'Type of Veh').find_parent('tr').find_all('td')[1].text
        except:
            veh_type = ''
            
        # 22. Car Category - e.g. 'low mileage', 'direct owner' and 'auction'. Each car can belong to multiple categories.
        ### categories: COE, OPC, Rare/exotic, low mileage, almost new, direct owner, PARF, STA evaluated, hybrid, auction, imported used, premium ad, consignment
        try:
            cat = main_left.find('strong', text = 'Category').find_parent('tr').find_all('td')[1].text
        except:
            cat = ''
            
        # 23. Date Posted
        date = main_left.find('div', id = 'usedcar_postdate').text
        date = re.findall(r"(?<=Posted on: )\w+-\w+-\w+",date)[0]
        date = datetime.strptime(date,'%d-%b-%Y')

        # 24. Date Updated - date updated is the date on which the seller has edited the listing. Not to be confused with the date on which the vehicle data in our local database was updated by the script.
        try:
            updated = main_left.find('div', id = 'usedcar_postdate').text
            updated = re.findall(r"(?<=Updated on: )\w+-\w+-\w+",updated)[0]
            updated = datetime.strptime(updated,'%d-%b-%Y')
        except:
            updated = 0
            
        # 25. Tags - contains permutations of the make, model and manufacturing year. E.g. "Mazda 3, mazda 3, 2014 Mazda 3, 2014 mazda 3, Mazda, Mazda 3, mazda 3, Used Mazda", with tags separated with commas.
        try:
            tags = main_left.find('div', id = 'cartags').text.replace('Tags: ','')
        except:
            tags = ''
        
        # 26. Date Sold - To initialise as 0; not scraped from webpage html.
        date_sold = 0
        
        # 27. Last Edited - To initialise as 0; not scraped from webpage html. Refers to the date on which the script had updated the car's data in our local database. 
        last_edited = 0

        return([int(car_id), avail, make_model, price, depre, reg_date, manu_yr, mileage, tranny, eng_cap, road_tax, power, weight, features, acc, desc,
              coe_left, coe_price, omv, arf, owners, veh_type, cat, date, updated, tags, date_sold, last_edited])
    else:
        return None

In [ ]:
# We then define a function update() that scrapes data for listings that have already been previously scraped using the process_new() function.
# The list of car_id's that will be run through this function will be obtained from our local DB using an SQL query. The SQL query will only extract the IDs of listings that are still available for sale (with avail = 'Available' in the local DB).
# The logic of the update() function is similar to that of the process_new() function, with a couple of differences which will be explained.

def update(car_id):
    
    # Similar to process_new(), we will grab the html as well as the title of the webpage for the specific car_id.
    url = 'https://www.sgcarmart.com/used_cars/info.php?ID=' + car_id
    response = get(url)
    page_html = BeautifulSoup(response.text,'html.parser')
    page_title = page_html.find('title').text
    
    # Again we extract the main_left section of the html that contains all of the key data that we need.
    try:
        main_left = page_html.find('div', attrs = {'id':'main_left'})
       ### tr_list = main_left.find_all('tr')
    except:
        None
    
    # Here we obtain the vehicle's availability status. 
    try:
        avail = str(main_left.find('strong', text = 'Availability').find_parent('tr').find_all('td')[1].text.replace('\n',''))
    except:
        avail = None
    
    # We initialise a date_sold variable prior to the following if statement.
    date_sold = 0
    
    # In the first scenario in which the vehicle was sold, we note down the date it was sold.
    # When the vehicle is sold, the seller has to update the listing and change the status of the car to 'Sold'.
    # We compare the date on which the listing was updated (updated) to the date the update() function was run (date_today).
    # If updated < date_today, the vehicle was sold before today and after the last time we updated the vehicle in the DB.
    # Else, the vehicle was sold today.
    if avail == 'SOLD':
        try:
            updated = main_left.find('div', id = 'usedcar_postdate').text
            updated = re.findall(r"(?<=Updated on: )\w+-\w+-\w+",updated)[0]
            updated = datetime.strptime(updated,'%d-%b-%Y').date()
            date_today = datetime.now().date()
            if updated < date_today:
                date_sold = updated
            else:
                date_sold = date_today
        except:
            date_sold = datetime.now().date()

        # Note the intentional placement of date_sold before car_id. This comes in useful later when we are writing changes to the DB.
        return([date_sold, int(car_id)])
    
    # Else if the vehicle is still available for sale, we will update all of its details.   
    elif avail == 'Available':

        # 1. Make and Model - e.g. Toyota Vios
        ### mm_repl_list = ['New','10-yr','COE',' (  )', '5-yr']
        make_model = page_html.find('a', class_ = 'link_redbanner').text
        ### for item in mm_repl_list:
        ###     make_model = make_model.replace(item, '')

        # 2. Price
        try:
            price = int(main_left.find('strong', text = 'Price').find_parent('tr').find_all('strong')[1].text.replace('$','').replace(',',''))
        except:
            price = 0
            
        # 3. Annual Depreciation
        ### depre_repl_list = ['$', ',', ' /yr']
        try:
            depre = int(main_left.find('strong', text = 'Depreciation').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',','').replace(' /yrView models with similar depreciation',''))
        except:
            depre = 0
        ### for item in depre_repl_list:
        ###     depre = depre.replace(item,'')
        ### depre = int(depre)

        # 4. Registration Date
        ### regdate_repl_list = 
        try:
            reg_date = main_left.find('strong', text = 'Reg Date').find_parent('tr').find_all('td')[1].text
            reg_date = re.findall(r"\d+.+\(", reg_date)[0].replace('(','')
            reg_date = datetime.strptime(reg_date,'%d-%b-%Y')
        except:
            reg_date = 0

        # 5. Year of Manufacture 
        try:
            manu_yr = int(main_left.find('strong', text = 'Manufactured').find_parent('tr').find_all('td')[1].text)
        except:
            manu_yr = 0
            
        # 6. Mileage (in kilometres)
        try:
            mileage = main_left.find('strong', text = 'Mileage').find_parent('tr').find_all('td')[1].text
            mileage = int(re.findall(r"\d+.\d+ k", mileage)[0].replace(' k','').replace(',',''))
        except:
            mileage = 0
            
        # 7. Transmission - Auto or Manual
        try:
            tranny = main_left.find('strong', text = 'Transmission').find_parent('tr').find_all('td')[1].text
        except:
            tranny = ''
            
        # 8. Engine Capacity (in cc)
        try:
            eng_cap = int(main_left.find('strong', text = 'Engine Cap').find_parent('tr').find_all('td')[1].text.replace(',','').replace(' cc',''))
        except:
            eng_cap = 0
            
        # 9. Road Tax / Year
        try:
            road_tax = int(main_left.find('strong', text = 'Road Tax').find_parent('tr').find_all('td')[1].text.replace(',','').replace('$','').replace(' /yr',''))
        except:
            road_tax = ''

        # 10. Power (in bhp)
        try:
            power = main_left.find('strong', text = 'Power').find_parent('tr').find_all('td')[1].text
            power = int(re.findall(r"\d+ bhp", power)[0].replace(' k','').replace(' bhp',''))
        except:
            power = ''

        # 11. Curb Weight (in kg)
        try:
            weight = int(main_left.find('strong', text = 'Curb Weight').find_parent('tr').find_all('td')[1].text.replace(',','').replace(' kg',''))
        except:
            weight = 0
            
        # 12. Features - descriptive field, e.g. 'reverse camera and dual zone climate control'; chunk of text and not tags
        try:
            features = main_left.find('strong', text = 'Features').find_parent('tr').find_all('td')[1].text
        except:
            features = ''
            
        # 13. Accessories - descriptive field, e.g. 'carbon fibre spoiler and custom bodykit'; chunk of text and not tags
        try:
            acc = main_left.find('a', text = 'Accessories').find_parent('tr').find_all('td')[1].text
        except:
            acc = ''
            
        # 14. Description - descriptive field, e.g. 'accident free and parked under shelter'; chunk of text and not tags
        try:
            desc = main_left.find('strong', text = 'Description').find_parent('tr').find_all('td')[1].text
        except:
            desc = ''
            
        # 15. COE Period Left (in days) - number of days in a month taken to be 30
        try:
            coe_left = main_left.find('strong', text = 'Reg Date').find_parent('tr').find_all('td')[1].text
            try:
                yrs = int(re.findall(r"\d+(?=yr)",coe_left)[0])
            except:
                yrs = 0
            try:
                mths = int(re.findall(r"\d+(?=mth)",coe_left)[0])
            except:
                mths = 0
            try:
                days = int(re.findall(r"\d+(?=day)",coe_left)[0])
            except:
                days = 0
            coe_left = yrs * 365 + mths * 30 + days
        except:
            coe_left= 0
            
        # 16. COE Price Paid
        try:
            coe_price = int(main_left.find('a', text = 'COE').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',',''))
        except:
            coe_price = 0
            
        # 17. Open Market Value (OMV)
        try:
            omv = int(main_left.find('strong', text = 'OMV').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',',''))
        except:
            omv = 0
            
        # 18. Additional Registration Fee (ARF)
        try:
            arf = int(main_left.find('strong', text = 'ARF').find_parent('tr').find_all('td')[1].text.replace('$','').replace(',',''))
        except:
            arf = 0
            
        # 19. Number of Past Owners
        try:
            owners = int(main_left.find('strong', text = 'No. of Owners').find_parent('tr').find_all('td')[1].text)
        except:
            owners = 0
            
        # 20. Vehicle Type - e.g. 'Sports Car', 'Luxury Sedan'. Each car can only have one vehicle type.
        try:
            veh_type = main_left.find('strong', text = 'Type of Veh').find_parent('tr').find_all('td')[1].text
        except:
            veh_type = ''
        
        # 21. Car Category - e.g. 'low mileage', 'direct owner' and 'auction'. Each car can belong to multiple categories.
        ###REGEX category - COE, OPC, Rare/exotic, low mileage, almost new, direct owner, PARF, STA evaluated, hybrid, auction, imported used, premium ad, consignment
        try:
            cat = main_left.find('strong', text = 'Category').find_parent('tr').find_all('td')[1].text
        except:
            cat = ''
            
        # 22. Date Posted
        date = main_left.find('div', id = 'usedcar_postdate').text
        date = re.findall(r"(?<=Posted on: )\w+-\w+-\w+",date)[0]
        date = datetime.strptime(date,'%d-%b-%Y')

        # 23. Date Updated - date updated is the date on which the seller has edited the listing. Not to be confused with the date on which the vehicle data in our local database was updated by the script. 
        try:
            updated = main_left.find('div', id = 'usedcar_postdate').text
            updated = re.findall(r"(?<=Updated on: )\w+-\w+-\w+",updated)[0]
            updated = datetime.strptime(updated,'%d-%b-%Y')
        except:
            updated = 0
            
        # 24. Tags - contains permutations of the make, model and manufacturing year. E.g. "Mazda 3, mazda 3, 2014 Mazda 3, 2014 mazda 3, Mazda, Mazda 3, mazda 3, Used Mazda", with tags separated with commas.
        try:
            tags = main_left.find('div', id = 'cartags').text.replace('Tags: ','')
        except:
            tags = ''
        
        # 25. Date Sold - update into the DB as 0.
        date_sold = 0

        return([int(car_id), avail, make_model, price, depre, reg_date, manu_yr, mileage, tranny, eng_cap, road_tax, power, weight, features, acc,
              coe_left, coe_price, omv, arf, owners, veh_type, cat, date, updated, tags, date_sold, desc])
    else:
        return None

In [ ]:
print(update('769367'))

In [ ]:
###scrape new

# With the process_new() function defined, the function was then run on batches of listing IDs. 

# Declare a variable start_time as the current time.
start_time = time()

# Connect to the local DB
conn = sqlite3.connect('db.db')

# Create a Cursor object and use it to execute SQL commands.
c = conn.cursor()

# Declare a list results to which we append the output of valid new listings that were run through the process_new() function.
results = []

###range done 617000 - 788047

# Set the range of ID values to scrape for new vehicles. 
# Running this script on a Jupyter Notebook meant that the range_end of the previous scraping session is the range_start of the current scraping session.
# As for range_end, I did a manual search on the online marketplace, checked the top 15 listings for the highest ID value, and made sure it was the highest by checking if the listing with the next higher ID value was valid.
# Initially when scraping for new vehicles for the first time, I used a small batch size of ~2,000 to minimise time wasted when this operation was interrupted - especially by broken internet connection or the free VPN service running out of data.
range_start = 793556
range_end = 793904

# Randomise order of IDs to make scraping less suspicious to the server.
id_randomised = random.sample(range(range_start, range_end), range_end - range_start)

# Loop through id_randomised and execute process_new() on every iteration.
# Only non-empty outputs are appended to the results list.
# sleep(random.uniform(0.1,0.2)) ensures that we do not flood the server with requests by making the script pause here for a random duration between 0.1 and 0.2 seconds.
for car_id in id_randomised:
    output = process_new(str(car_id))
    if output != None:
        results.append(output)
    sleep(random.uniform(0.1,0.2))

# Print the results list to quickly scan for any abnormalities.
print(results)
# Print the number of new listings scraped.
print('New listings scraped: ' + str(len(results)))

# executemany() executes multiple lines of SQL code. my_iter was defined to generate every listing from the results list, in order for executemany() to process them.
def my_iter(x):
    for i in x:
        yield i
c.executemany("INSERT INTO cars VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", my_iter(results))

# Confirm the changes to be made to the DB
conn.commit()

# Close the connection to the DB
conn.close()

# Calculate time taken for the whole operation
elapsed = round(time() - start_time, 2)
print("time elapsed = " + str(elapsed))
print("time taken per page = " + str(round(elapsed / (range_end-range_start), 2)))

In [ ]:
# Generate list of IDs of listings that should be updated - i.e. listings that still have 'Available' as their status.

# Declare a variable start_time as the current time.
start_time = time()

# Connect to the local DB
conn = sqlite3.connect('db.db')

# Create a Cursor object and use it to execute SQL commands.
c = conn.cursor()

date_today = datetime.now().date()
### date_today = datetime.now().date() - dt.timedelta(days = 1)

# SELECT from DB only vehicles that are still available for sale, and have not been edited today.
avail_cars = c.execute("SELECT id FROM cars where avail = 'Available' and last_edited is not ?", [date_today])

# For some reason the output of the above SQL query returns a tuple of tuples e.g. ((700000,), (700001,), (700002,)).
# Hence in the next 3 lines we extract the elements from the tuples and append them to a new, neat list named avail_id.
avail_id = []
for row in avail_cars:
    avail_id.append(row[0])

# Break up the list of IDs into smaller batch sizes for better processing later on
update_dictionary={}
batch_size = 1500
for x in range(1,15):
    try:
        update_dictionary["batch{0}".format(x)] = avail_id[((x-1)*batch_size):(x*batch_size)]
    except:
        update_dictionary["batch{0}".format(x)] = []

# Confirm the changes to be made to the DB
conn.commit()

# Close the connection to the DB
conn.close()

# Print the count of cars to update and the whole of update_dictionary
print("Number of cars to update: " + str(len(avail_id)))
print(update_dictionary)

In [ ]:
# Begin updating by batches

# Iterate through every batch in update_dictionary. Should this operation get interrupted, re-run the previous step to generate the latest list of IDs to be updated, and run this step again.
for key in update_dictionary:
    
    # Declare a variable start_time as the current time. To calculate time taken to process every single batch.
    start_time = time()

    # Declare a list to which we append the output from running the update() function
    results = []

    # Obtain the list of IDs corresponding to the key in the current iteration
    batch = update_dictionary[key]

    # Randomise sorting of IDs to make scraping less suspicious to the server
    id_randomised = random.sample(batch, len(batch))

    # Perform update() function on all car IDs in the list; pause for a random period of time to reduce suspicion and the frequency of get requests
    for car_id in id_randomised:
        output = update(str(car_id))
        if output != None:
            results.append(output)
        sleep(random.uniform(0.05,0.1))

    # Declare a list sold_list to which sold listings will be appended
    sold_list = []

    # Declare a list update_list to which available listings will be appended; the latest listing data will still be overwritten into the DB
    update_list = []
    
    # Using the second element in the list output of update(), we sort the sold listings and available listings into the respective lists.
    # Sold listings have the list element in index 1 (second element) set as the listing ID (int(car_id)) with type() = int, while available listings have it set as the vehicle availability (avail) with type() = str.
    for result in results:
        if type(result[1]) == int:
            sold_list.append(result)
        elif type(result[1]) == str:
            update_list.append(result)
    
    # Print sold_list, update_list and their respective lengths for quick checking 
    print(sold_list)
    print('Length of sold_list: ' + str(len(sold_list)))
    print(update_list)
    print('Length of update_list: ' + str(len(update_list)))

    # Connect to the DB
    conn = sqlite3.connect('db.db')
    
    # Create a Cursor object and use it to execute SQL commands.
    c = conn.cursor()
    ###stopped here; 2 lines of executemany for sold_list and update_list        

    # Execute to DB updates for sold vehicles
    try:
        for car in sold_list:
            c.execute("UPDATE cars SET avail=?, date_sold=?, last_edited=? WHERE id=?", ['Sold',car[0],datetime.now().date(),car[1]])

    except:
        None

    # Execute to DB updates for available vehicles. Very tedious method used in matching the list elements to their respective columns in the DB. Was not able to find a simplified way which worked...
    for car in update_list:
        c.execute('''UPDATE cars SET make_model=?, price=?, depre=?, reg_date=?, manu_yr=?, mileage=?, tranny=?, eng_cap=?, 
        road_tax=?, power=?, weight=?, features=?, acc=?, coe_left=?, coe_price=?, omv=?, arf=?, owners=?, veh_type=?,
        cat=?, date_posted=?, updated=?, tags=?, last_edited=?, desc=? WHERE id=?''',[car[2],car[3],car[4],car[5],car[6],car[7],car[8],car[9],car[10],
                                                          car[11],car[12],car[13],car[14],car[15],car[16],car[17],car[18],
                                                          car[19],car[20],car[21],car[22],car[23],car[24],datetime.now().date(),car[26],car[0]])
    
    # Confirm the changes to be made to the DB
    conn.commit()

    # Close the connection to the DB
    conn.close()

    # Calculate time taken for the whole operation
    elapsed = round(time() - start_time, 2)
    print("time elapsed = " + str(elapsed))
    print("time taken per page = " + str(round(elapsed/(len(batch)),2)))

### os.system('shutdown -s')

In [ ]:
for key in update_dictionary:
    print(update_dictionary[key])

In [ ]:
print(update('787754'))

In [ ]:
def my_iter(x):
    for i in x:
        yield i

In [ ]:
test = [1,2,3]
print(my_iter(test))

In [ ]:
# Generate list of IDs of listings that should be updated - i.e. listings that still have 'Available' as their status.

# Declare a variable start_time as the current time.
start_time = time()

# Connect to the local DB
conn = sqlite3.connect('db.db')

# Create a Cursor object and use it to execute SQL commands.
c = conn.cursor()

date_today = datetime.now().date()
### date_today = datetime.now().date() - dt.timedelta(days = 1)

# SELECT from DB only vehicles that are still available for sale, and have not been edited today.
avail_cars = c.execute("SELECT id FROM cars where avail = 'Available' and last_edited is not ?", [date_today])
for row in avail_cars:
    print(row)

# avail_id = []

# for row in avail_cars:
#     avail_id.append(row[0])

# update_dictionary={}
# batch_size = 1500
# for x in range(1,9):
#     try:
#         update_dictionary["batch{0}".format(x)] = avail_id[((x-1)*batch_size):(x*batch_size)]
#     except:
#         update_dictionary["batch{0}".format(x)] = []
        
# conn.commit()

# conn.close()

# print("Number of cars to update: " + str(len(avail_id)))
# print(update_dictionary)

In [ ]:
for i in ((1,),(2,),(3,)):
    print(i[0]**2)

In [ ]:
### Rescrape coe_left

from requests import get
from bs4 import BeautifulSoup
from time import sleep
from time import time
import random
import functools
import sqlite3
from sqlite3 import Error
import re
from datetime import datetime
import datetime as dt
import os

def update_coe(car_id):
    
    # Similar to process_new(), we will grab the html as well as the title of the webpage for the specific car_id.
    url = 'https://www.sgcarmart.com/used_cars/info.php?ID=' + car_id
    response = get(url)
    page_html = BeautifulSoup(response.text,'html.parser')
    page_title = page_html.find('title').text
    
    # Again we extract the main_left section of the html that contains all of the key data that we need.
    try:
        main_left = page_html.find('div', attrs = {'id':'main_left'})
       ### tr_list = main_left.find_all('tr')
        try:
            coe_left = main_left.find('strong', text = 'Reg Date').find_parent('tr').find_all('td')[1].text
            try:
                yrs = int(re.findall(r"\d+(?=yr)",coe_left)[0])
            except:
                yrs = 0
            try:
                mths = int(re.findall(r"\d+(?=mth)",coe_left)[0])
            except:
                mths = 0
            try:
                days = int(re.findall(r"\d+(?=day)",coe_left)[0])
            except:
                days = 0
            coe_left = yrs * 365 + mths * 30 + days
        except:
            coe_left = 0
    except:
        None

    return([int(car_id), coe_left])

In [ ]:
update_coe('772131')

In [ ]:
# Generate list of IDs of listings that should be updated - i.e. listings that still have 'Available' as their status.

# Declare a variable start_time as the current time.
start_time = time()

# Connect to the local DB
conn = sqlite3.connect('db.db')

# Create a Cursor object and use it to execute SQL commands.
c = conn.cursor()

date_today = datetime.now().date()
### date_today = datetime.now().date() - dt.timedelta(days = 1)

# SELECT from DB only vehicles that are still available for sale, and have not been edited today.
avail_cars = c.execute("select * from cars where coe_left not in (1825, 3650) and recalc_coe_left=0 ORDER BY last_edited asc")
# avail_cars = c.execute("select * from cars where id in (772131, 772540, 773178, 772886)")

# For some reason the output of the above SQL query returns a tuple of tuples e.g. ((700000,), (700001,), (700002,)).
# Hence in the next 3 lines we extract the elements from the tuples and append them to a new, neat list named avail_id.
avail_id = []
for row in avail_cars:
    avail_id.append(row[0])

# Break up the list of IDs into smaller batch sizes for better processing later on
update_dictionary={}
batch_size = 1500
for x in range(1,15):
    try:
        update_dictionary["batch{0}".format(x)] = avail_id[((x-1)*batch_size):(x*batch_size)]
    except:
        update_dictionary["batch{0}".format(x)] = []

# Confirm the changes to be made to the DB
conn.commit()

# Close the connection to the DB
conn.close()

# Print the count of cars to update and the whole of update_dictionary
print("Number of cars to update: " + str(len(avail_id)))
print(update_dictionary)

In [ ]:
# Begin updating by batches

# Iterate through every batch in update_dictionary. Should this operation get interrupted, re-run the previous step to generate the latest list of IDs to be updated, and run this step again.
for key in update_dictionary:
    
    # Declare a variable start_time as the current time. To calculate time taken to process every single batch.
    start_time = time()

    # Declare a list to which we append the output from running the update() function
    results = []

    # Obtain the list of IDs corresponding to the key in the current iteration
    batch = update_dictionary[key]

    # Randomise sorting of IDs to make scraping less suspicious to the server
    id_randomised = random.sample(batch, len(batch))

    # Perform update() function on all car IDs in the list; pause for a random period of time to reduce suspicion and the frequency of get requests
    for car_id in id_randomised:
        output = update_coe(str(car_id))
        if output != None:
            results.append(output)
        sleep(random.uniform(0.01,0.03))

    # Connect to the DB
    conn = sqlite3.connect('db.db')
    
    # Create a Cursor object and use it to execute SQL commands.
    c = conn.cursor()
    ###stopped here; 2 lines of executemany for sold_list and update_list        

    # Execute to DB updates for sold vehicles
    try:
        for car in results:
            c.execute("UPDATE cars SET recalc_coe_left=? WHERE id=?", [car[1],car[0]])

    except:
        None
    
    # Confirm the changes to be made to the DB
    conn.commit()

    # Close the connection to the DB
    conn.close()

    try:
    # Calculate time taken for the whole operation
        elapsed = round(time() - start_time, 2)
        print(len(batch))
        print("time elapsed = " + str(elapsed))
        print("time taken per page = " + str(round(elapsed/(len(batch)),2)))
        print(results)
    
    except:
        None

### os.system('shutdown -s')

In [ ]:
### Rescrape availability

from requests import get
from bs4 import BeautifulSoup
from time import sleep
from time import time
import random
import functools
import sqlite3
from sqlite3 import Error
import re
from datetime import datetime
import datetime as dt
import os

def update_avail(car_id):
    
    # Similar to process_new(), we will grab the html as well as the title of the webpage for the specific car_id.
    url = 'https://www.sgcarmart.com/used_cars/info.php?ID=' + car_id
    response = get(url)
    page_html = BeautifulSoup(response.text,'html.parser')
    page_title = page_html.find('title').text
    
    # Again we extract the main_left section of the html that contains all of the key data that we need.
    try:
        main_left = page_html.find('div', attrs = {'id':'main_left'})
       ### tr_list = main_left.find_all('tr')
        try:
            avail = str(main_left.find('strong', text = 'Availability').find_parent('tr').find_all('td')[1].text.replace('\n',''))
        except:
            avail = 0
    except:
        None

    return([int(car_id), avail])

In [ ]:
update_avail('762651')

In [ ]:
# Generate list of IDs of listings that should be updated - i.e. listings that still have 'Available' as their status.

# Declare a variable start_time as the current time.
start_time = time()

# Connect to the local DB
conn = sqlite3.connect('extracted db.db')

# Create a Cursor object and use it to execute SQL commands.
c = conn.cursor()

date_today = datetime.now().date()
### date_today = datetime.now().date() - dt.timedelta(days = 1)

# SELECT from DB only vehicles that are still available for sale, and have not been edited today.
avail_cars = c.execute("select * from cars where avail is null")
# avail_cars = c.execute("select * from cars where id in (772131, 772540, 773178, 772886)")

# For some reason the output of the above SQL query returns a tuple of tuples e.g. ((700000,), (700001,), (700002,)).
# Hence in the next 3 lines we extract the elements from the tuples and append them to a new, neat list named avail_id.
avail_id = []
for row in avail_cars:
    avail_id.append(row[0])

# Break up the list of IDs into smaller batch sizes for better processing later on
update_dictionary={}
batch_size = 1500
for x in range(1,15):
    try:
        update_dictionary["batch{0}".format(x)] = avail_id[((x-1)*batch_size):(x*batch_size)]
    except:
        update_dictionary["batch{0}".format(x)] = []

# Confirm the changes to be made to the DB
conn.commit()

# Close the connection to the DB
conn.close()

# Print the count of cars to update and the whole of update_dictionary
print("Number of cars to update: " + str(len(avail_id)))
print(update_dictionary)

In [ ]:
# Begin updating by batches

# Iterate through every batch in update_dictionary. Should this operation get interrupted, re-run the previous step to generate the latest list of IDs to be updated, and run this step again.
for key in update_dictionary:
    
    # Declare a variable start_time as the current time. To calculate time taken to process every single batch.
    start_time = time()

    # Declare a list to which we append the output from running the update() function
    results = []

    # Obtain the list of IDs corresponding to the key in the current iteration
    batch = update_dictionary[key]

    # Randomise sorting of IDs to make scraping less suspicious to the server
    id_randomised = random.sample(batch, len(batch))

    # Perform update() function on all car IDs in the list; pause for a random period of time to reduce suspicion and the frequency of get requests
    for car_id in id_randomised:
        output = update_avail(str(car_id))
        if output != None:
            results.append(output)
        sleep(random.uniform(0.01,0.03))
    
    end_scrape = time()

    # Connect to the DB
    conn = sqlite3.connect('extracted db.db')
    
    # Create a Cursor object and use it to execute SQL commands.
    c = conn.cursor()
    ###stopped here; 2 lines of executemany for sold_list and update_list        

    # Execute to DB updates for sold vehicles
    try:
        for car in results:
            c.execute("UPDATE cars SET avail=? WHERE id=?", [car[1],car[0]])

    except:
        None
    
    # Confirm the changes to be made to the DB
    conn.commit()

    # Close the connection to the DB
    conn.close()

    try:
    # Calculate time taken for the whole operation
        
        elapsed = round(time() - start_time, 2)
        elapsed_scrape = round(end_scrape - start_time, 2)
        print(len(batch))
        print("time elapsed = " + str(elapsed))
        print("time taken per page to scrape = " + str(round(elapsed_scrape/(len(batch)),2)))
        print("total time taken per page = " + str(round(elapsed/(len(batch)),2)))
        print(results)
    
    except:
        None

### os.system('shutdown -s')